conv->relu->pool->flat->fc

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

batch_size = 100
n_batch = mnist.train.num_examples // batch_size

def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x,W):
    # input [batch,in_height, in_width, in_channels]
#     tf.nn.conv2d(
#         input,
#         filter,
#         strides,
#         padding,
#         use_cudnn_on_gpu=True,
#         data_format='NHWC',
#         dilations=[1, 1, 1, 1],
#         name=None,
#     )
    # x input 4-D [batch,in_height, in_width,in_channels]
    # w weights 4-D [filter_height,filter_width,in_channels, out_channels]
    # strides[0]=strides[3]=1 strides[0]表示x方向的步长，strides[1]表示y方向的步长
    # padding： SAME   VALID
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    # ksize=[1,x,y,1] 2x2的池化层  strides = [1,x,y,1] x:步长2  y：步长2
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

# x input  [batch,in_height,in_width,in_channels]
x_image = tf.reshape(x,[-1,28,28,1])

#w        [filter_height,filter_width,in_channels, out_channels]
W_conv1 = weight_variable([5,5,1,32]) # 5*5的采样窗口，32个卷积核从1个平面抽取特征
b_conv1 = bias_vairable([32]) #每个卷积核一个偏置值

# 28*28*1 的图片卷积之后变为28*28*32
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
# 池化之后变为 14*14*32
h_pool1 = max_pool_2x2(h_conv1)

# 第二次卷积之后变为 14*14*64
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_vairable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)

# 第二次池化之后变为 7*7*64
h_pool2 = max_pool_2x2(h_conv2)


# 第一个全连接层
W_fc1 = weight_variable([7*7*64,1024]) #上一层有7*7*64个神经元，全连接层有1024个神经元
b_fc1 = bias_vairable([1024])#1024个节点

# 把池化层2的输出扁平化为1维向量
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
#求第一个全连接的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 初始化第二个全连接层
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_vairable([10])
logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2

#计算输出
prediction = tf.nn.sigmoid(logits)

#交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
#AdamOptimizer
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

prediction_2 = tf.nn.softmax(prediction)
correct_prediction = (tf.equal(tf.argmax(prediction_2,1), tf.argmax(y,1)))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})
        print("Iter: " + str(epoch) + ", acc: " + str(acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input

## 重构代码

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
batch_size = 128
n_batch = mnist.train.num_examples // batch_size
max_step = 1000
keep_ = 0.8
log_dir = "Logs/log-6.1"

# 生成权重
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1),name='W')

# 生成偏差
def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape),name='b')

# 记录变量
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME',name='conv2d')
        
def conv_layer(input_tensor, weight_shape, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable(weight_shape)
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_vairable([weight_shape[-1]])
            variable_summaries(biases)
        with tf.name_scope('conv_comput'):
            preactivate = conv2d(input_tensor,weights) + biases
        with tf.name_scope('activate'):
            activations = act(preactivate)
        return activations

def linear_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_vairable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('linear_comput'):
            preactivate = tf.matmul(input_tensor,weights) + biases
        with tf.name_scope('activate'):
            activations = act(preactivate)
        return activations
        

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Max_pool')

with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32,[None,784],name='input_x')
    with tf.name_scope('Input_reshape'):
        x_image = tf.reshape(x,[-1,28,28,1],name='x-image')
        tf.summary.image('input',x_image,10)
    y = tf.placeholder(tf.float32,[None,10],name='input_y')
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')

# 第一次卷积   28*28*1->28*28*32
conv_layer1 = conv_layer(x_image,[5,5,1,32],'conv_layer1')
# 池化之后变为 14*14*32
with tf.name_scope('Max_pool1'):
    h_pool1 = max_pool_2x2(conv_layer1)

# 第二次卷积 14*14*32->14*14*64
conv_layer2 = conv_layer(h_pool1,[5,5,32,64],'conv_layer2')
# 第二次池化之后变为 7*7*64
with tf.name_scope('Max_pool2'):
    h_pool2 = max_pool_2x2(conv_layer2)

with tf.name_scope('Flatten'):
    flatten_ = tf.reshape(h_pool2,[-1,7*7*64])
    
# 第一个全连接层 7*7*64 - 1024
fc1 = linear_layer(flatten_, 7*7*64, 1024, 'FC1')

with tf.name_scope('Dropput'):
    fc1_drop = tf.nn.dropout(fc1, keep_prob)
    
# 第二个全连接层 1024 - 10
logits = linear_layer(fc1_drop, 1024, 10, 'FC2',act=tf.nn.sigmoid)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

with tf.name_scope('accuracy'):
    prediction = tf.nn.softmax(logits)
    correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    
merged = tf.summary.merge_all()

def get_dict(train):
    if train:
        xs, ys = mnist.train.next_batch(batch_size)
        k = keep_
    else:
        xs, ys = mnist.test.images, mnist.test.labels
        k = 1.0
    return {x:xs, y:ys, keep_prob: k}

with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(max_step):
        if i%10 == 0:
            summary,acc = sess.run([merged,accuracy], feed_dict=get_dict(False))
            test_writer.add_summary(summary, i)
            print("Step: " + str(i) + ", acc: " + str(acc))
        else:
            summary,_ = sess.run([merged,train_step], feed_dict=get_dict(True))
            train_writer.add_summary(summary,i)
        
    train_writer.close()
    test_writer.close()

Step: 0, acc: 0.11
Step: 10, acc: 0.138
Step: 20, acc: 0.1994
Step: 30, acc: 0.3193
Step: 40, acc: 0.2373
Step: 50, acc: 0.2421
Step: 60, acc: 0.527
Step: 70, acc: 0.7946
Step: 80, acc: 0.7877
Step: 90, acc: 0.8439
Step: 100, acc: 0.8235
Step: 110, acc: 0.8555
Step: 120, acc: 0.8639
Step: 130, acc: 0.8655
Step: 140, acc: 0.8672
Step: 150, acc: 0.8711
Step: 160, acc: 0.8726
Step: 170, acc: 0.8754
Step: 180, acc: 0.8746
Step: 190, acc: 0.8771
Step: 200, acc: 0.8782
Step: 210, acc: 0.8805
Step: 220, acc: 0.8806
Step: 230, acc: 0.8763
Step: 240, acc: 0.8815
Step: 250, acc: 0.8805
Step: 260, acc: 0.877
Step: 270, acc: 0.8811
Step: 280, acc: 0.8824
Step: 290, acc: 0.8827
Step: 300, acc: 0.8829
Step: 310, acc: 0.8843
Step: 320, acc: 0.8853
Step: 330, acc: 0.8838
Step: 340, acc: 0.8837
Step: 350, acc: 0.8833
Step: 360, acc: 0.8853
Step: 370, acc: 0.885
Step: 380, acc: 0.8841
Step: 390, acc: 0.8871
Step: 400, acc: 0.8871
Step: 410, acc: 0.8877
Step: 420, acc: 0.8874
Step: 430, acc: 0.8888
Step: